In [1]:
import pytesseract
import cv2
import regex as re

In [2]:
row_length = 8

In [3]:
img = cv2.imread('resources/osmismerka.jpg')
baseImg = img.copy()

# Preprocessing
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7, 7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
dilated = cv2.dilate(thresh, kernel, iterations=1)

In [4]:
# Find contours
cnts = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

# Group contours into rows
def group_contours_by_rows(contours, row_threshold=10):
    rows = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        added = False
        for row in rows:
            # Check if the contour belongs to an existing row
            if abs(row[0][1] - y) < row_threshold:
                row.append((x, y, w, h, c))
                added = True
                break
        if not added:
            # Create a new row if no existing row matches
            rows.append([(x, y, w, h, c)])
    return rows

# Group contours into rows and sort them
rows = group_contours_by_rows(cnts)
rows = sorted(rows, key=lambda row: row[0][1])

# Sort contours in each row by x-coordinate
sorted_contours = []
for row in rows:
    row = sorted(row, key=lambda item: item[0])
    sorted_contours.extend([item[4] for item in row])

letters = []

# Extract letters
for c in sorted_contours:
    x, y, w, h = cv2.boundingRect(c)
    letters.append(img[y-3:y+3 + h, x-3:x+3 + w])
    # cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display letters
# for i, letter in enumerate(letters):
#     cv2.imshow('Letter', letter)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [5]:
# Highlight the find in the original image
def draw_line (baseImg, contour1, contour2):
    x1, y1, w1, h1 = cv2.boundingRect(contour1)
    x2, y2, w2, h2 = cv2.boundingRect(contour2)
    cv2.line(baseImg, (int(x1 + 0.5 * w1), int(y1 + 0.5 * h1)), (int(x2 + 0.5 * w2), int(y2 + 0.5 * h2)), (0, 0, 255), 3, lineType=cv2.LINE_AA)
    cv2.imshow('Letter', baseImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
# draw_line(baseImg.copy(), sorted_contours[0], sorted_contours[10])

In [7]:
text = ''
row_number = round(len(letters) / row_length)
letterArray = [[0 for x in range(row_length)] for y in range(row_number)]
row = 0

for i, letter in enumerate(letters):
    if i % row_length == 0 and i != 0:
        text += '\n'
        row += 1
    add = pytesseract.image_to_string(letter, lang='eng', config='--psm 10').strip()
    if '|' in add:
        add = 'I'
    elif '(' in add:
        add = 'C'
    elif len(add) > 1:
        add = add[0].upper()
    else:
        add = add.upper()
    letterArray[row][i % row_length] = add
    text += add + ' '

print(text)

L L C E I M B K 
E R O B O T A E 
T Y H D F K A D 
A Z C S N K U I 
D T P E T X I V 
L N N S K R V D 
O A O V L A K E 
P K A U T O B M 


In [8]:
def transpose_array(array):
    return [list(row) for row in zip(*array)]

In [9]:
def get_index_of_letter(coordinates):
    return coordinates[0] + coordinates[1] * row_length

In [10]:
# Returns the indexes of the first and last letter of the word in the array
# Can be used for searching vertically when the array is transposed
def horizontal_array_search(array, word, transposed):
    regex = re.compile(word)
    for i in range(row_number):
        concat = ''
        
        # Create a string from row
        for j in range(row_length):
            concat += array[i][j]
        
        # Search forward
        match = re.search(regex, concat)
        if match:
            if transposed:
                return (get_index_of_letter((i, match.start())), get_index_of_letter((i, match.end() - 1)))
            return (get_index_of_letter((match.start(), i)), get_index_of_letter((match.end() - 1, i)))
        
        # Search backward
        concat = concat[::-1]
        match = re.search(regex, concat)
        if match:
            if transposed:
                return (get_index_of_letter((i, row_length - match.end())), get_index_of_letter((i, row_length - match.start() - 1)))
            return (get_index_of_letter((row_length - match.end(), i)), get_index_of_letter((row_length - match.start() - 1, i)))

In [11]:
def search(word):
    word = word.upper()
    
    # Search horizontally
    indexes = horizontal_array_search(letterArray, word, False)
    
    # Search vertically
    if indexes is None:
        letterArrayT = transpose_array(letterArray)
        indexes = horizontal_array_search(letterArrayT, word, True)
    
    # Highlight result
    if indexes is not None:
        draw_line(baseImg.copy(), sorted_contours[indexes[0]], sorted_contours[indexes[1]])
    else:
        print("Word not found")

In [14]:
search("ROBOT")